트리의 앙상블

- 정형 데이터(structured data): 어떤 구조로 되어 있음. csv, excel에 저장하기 쉬움.
- 비정형 데이터(untructured data): DB나 엑셀로 표현하기 어려운 것들. / 텍스트 데이터, 사진, 음악 등

- 그 동안 배운 머신러닝 알고리즘 : 정형 데이터에 잘 맞음
-  앙상블 학습(ensemble learning) = 정형 데이터를 다루는 데 가장 뛰어난 상과를 내는 알고리즘.


랜덤 포레스트(Random Forest)

- 앙상블 학습의 대표 주자 중 하나
- 안정적인 성능 덕분에 널리 사용됨.

- 결정 트리를 랜덤하게 만들어 결정 트리(나무)의 숲을 만듦.
- 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만듦.

In [ ]:
# 랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 만듦.
# 입력한 훈련 데이터에서 랜덤하게 샘플 추출하여 훈련 데이터 만듦.
# 이때 한 샘플이 중복되어 추출될 수도 있음.

부트스트랩 샘플 bootstrap sample
- 중복된 값도 뽑아내는 샘플 방식
- 노드 분할 때 전체 특성 중 일부 특성을 무작위로 고른 다음 이 중 최선의 분할을 찾음
- 분류 모델인 RandomForestClassifier는 기본적으로 전체 특성 개수를 
 제곱근만큼의 특성 선택 (4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택해 사용)
- 회귀 모델인 RandomForestRegressor는 전체 특성을 사용

In [ ]:
# 사이킷 런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 이런 방식으로 훈련함.
# 그 다음 분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음.
# 회귀일 때는 단순히 각 트리의 예측 평균

In [ ]:
# 랜덤 포레스트

# 랜덤하게 선택한 샘플과 특성 사용
# >> 훈련 세트에 과대적합되는 것을 막아주고 검증 세으톼 테스트 세트에서 안정적인 성능을 얻을 수 있음
# >> 기본 매개변수 설정 만으로도 아주 좋은 결과를 냄.

In [1]:
# 사이킷런 RandomForestClassifier 클래스를 화이트 와인을 분류하는 문제에 적용해보기.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
# cross_validate() 이요해 교차검증 수행하기 
# return_train_score= True, 검증 점수 뿐 아니라 훈련 세트에 대한 점수도 반환
# 과대적합을 파악하기 용이.

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
scores

{'fit_time': array([0.26408887, 0.26684809, 0.25736117, 0.2770803 , 0.26865244]),
 'score_time': array([0.03565931, 0.04060221, 0.03321886, 0.0431931 , 0.03916502]),
 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]),
 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}

In [3]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 훈련 세트에 다소 과대적합된 것으로 보임.

0.9973541965122431 0.8905151032797809


In [4]:
rf.fit(train_input,train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [ ]:
# 이전 결정트리에서 특성 중요도는 당도에서 매우 높았는데 
# 이번에는 다른 특성들의 중요도가 상승함.
# >> 랜덤 포레스트는 특성의 일부를 랜덤하게 선택하여 결정트리를 훈련함
# >> 그 결과하나의 특성에 과도하게 집중하지 않고 좀 더 많은 튻어이 훈련에 기여할 기회를 얻음
# >> 과대적합을 줄이고 일반화 성능을 높이는데 도움이 됨.

RandomForestClassifier의 기능 : 자체적으로 모델을 평가하는 점수를 얻을 수 있음.

- 랜덤 포레스트는 훈련 세트에서 중복을 허용/ 부트스트랩 샘플을 만들어 결정트리 훈련
- 이때 부트스트랩 샘플에 포함되지 않고 남는 샘플 == OOB (out of bag)
- OOB를 사용하여 부트스트랩 샘플로 훈련한 결정트리 평가 >> 검증세트 역할

- 이 점수를 얻기 위해서 oob_score 매개변수 =True 지정해야함. 기본값 ==False
- oob_score = True, 각 결정 트리의 OOB 점수를 평균하여 출력

In [6]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


엑스트라 트리(extra tree)

- 랜덤 포레스트와 비슷하게 동장
- 차이점 : 부트스트랩을 사용하지 않음 >> 결정 트리를 만들 때 전체 훈련 세트를 사용
- 대신 노드 분할할 때 가장 좋은 분할을 찾지 않고 무작위로 분할함.
- 엑스트라 트리가 사용하는 결정 트리가 DecisionTreeClassifier 의 spliter매개변수를 'random'으로 지정한 것과 같음

In [7]:
# 하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지지만
# 많은 트리를 앙상블하기 때문에 과대적합을 막고 검증 세트 점수를 높이는 효과가 있음.

from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [8]:
# 중요도 확인 가능
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


그레이디언트 부스팅 (gradient boosting)
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
- 사이킷런의 GradientBoostingClassifier 는 기본적으로 깊이가 3인 결정 트리를 100개 사용.
- 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로는 높은 일반화 성능 기대 가능

- 경사 하강법을 사용하여 트리를 앙상블에 추가
- 분류에서는 로지스틱 함수 사용
- 회기에서는 평균 제곱 오차 함수 사용

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대적합되지 않은 점수가 나옴!
# 결정 트리 개수를 늘려도 과대적합 되지 않음.>> 학습률 증가 및 성능 향상

0.8881086892152563 0.8720430147331015


In [11]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [12]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68011572 0.16116151]


In [ ]:
# subsample 매개변수 : 트리 훈련에 사용할 훈련 세트의 비율을 정하는 매개변수
# 기본값 == 1, 전체 훈련 세트 사용.

# 일반적으로 그레디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 냄.
# 하지만 순서대로 트리를 추가하기 때문에 훈련 속도가 느림.
# 즉, GradientBoostingClassifier에는 n_jobs 매개변수가 없음

히스토그램 기반 그레이디언트 부스팅(Histogram-based gradient boosting)

- 정형 데이터를 다루는 머신러닝 중 가장 인기가 높은 알고리즘.
- 
- 우선 입력 특성을 256개의 구간으로 나눔. >> 노드를 분할할 때 최적의 분할을 빠르게 찾을 수 있음.
- 부스팅 256개의 구간 중 하나를 떼어 놓고 누락된 값을 위해 사용함.
- 입력에 누락된 특성이 있어도 이를 따로 전처리할 필요가 없음!
- 
- HistGradientBoostingClassifier()
- 트리의 개수를 지정하는데 n_estimators 대신 max_iter 사용

In [13]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

c:\Users\user\anaconda3\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [14]:
# 특성 중요도 확인
# 히스토그램 기반 그레디언트 부스팅의 특성 중요도 계산 // permutation_importance() 함수 사용
# >> 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지 관찰하여 어떤 특성이 중요한지 계산

# 훈련 세트뿐만 아니라 테스트 세트에도 적용 가능. 사이킷런에서 제공하는 모든 추정기 모델 훈련에 사용 가능


# 히스토그램 기반 부스팅 모델 훈련 후 훈련세트에서 특성 중요도 계산.
# n_repeats 매개변수는 랜덤하게 섞을 횟수 지정.


from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [15]:
# 테스트 세트에서 특성 중요도 계산.
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [16]:
# 최종적으로 성능 확인
hgb.score(test_input, test_target)

0.8723076923076923

XGBoost

In [23]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist',random_state=42)
scores = cross_validate(xgb,train_input, train_target, return_train_score=True)
scores
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [25]:
!pip install lightgbm

     ---------------------------------------- 1.0/1.0 MB 9.2 MB/s eta 0:00:00


In [27]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


코드 적어보기

In [ ]:
# 랜덤 포레스트

In [28]:
import pandas as pd
import numpy as np
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [30]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2)


In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
rf = RandomForestClassifier(n_jobs=-1)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9978353011309471 0.889741245280225


In [34]:
# OOB 뽑아내기.
rf = RandomForestClassifier(oob_score=True, n_jobs=-1)
rf.fit(train_input, train_target)
scores = cross_validate(rf, train_input, train_target,return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
print(rf.oob_score_)

0.9978834011790472 0.8883939808987933
0.898787762170483


In [36]:
# 엑스트라 트리
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1)
scores = cross_validate(et, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9978834011790472 0.8833925001850892


In [38]:
# 그레이디언트 부스팅
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
scores = cross_validate(gb, train_input, train_target, return_train_score= True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8858473389866219 0.8718449692751905


In [39]:
# 그레이디언트 부스팅 n_estimators = 

gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9432845355626208 0.871653587028948


In [41]:
# 히스토리그램 기반 그레이디언트 부스팅

from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist')
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.954011910807677 0.8766554379210779


In [ ]:
# ** 키워드 핵심 **

# ** 앙상블 학습 : 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘
# ** 랜덤 포레스트 : 대표적인 결정 트리 기반의 앙상블 학습.
#   부스스트랩 샘플을 사용하고 랜덤하게 특성을 선택하여 트리를 만드는 것이 특징
# ** 엑스트라 트리 : 랜덤 포레스트와 비슷하게 결정 트리를 사용하여 앙상블 모델을 만들지만
#   부트스트랩은 사용하지 않음. 대신 랜덤하게 노드를 분할해 과대적합 감소
# ** 그레이디언트 부스팅 : 결정 트리를 연속적으로 추가하여 손실함 수 최소화하는 앙상블 방법
#     훈련 속도가 조금 느리지만 더 좋은 성능 기대
  # ** 히스토그램 기반 그레이디언트 부스팅 : 그레이디언트 부스팅 속도 개선

In [ ]:
# ** 패키지 함수 **

# ** RandomForestClassifier 랜덤 포레스트 분류 클래스
#   * n_estimators 매개변수 : 앙상블을 구성할 트리의 개수 지정. 기본값 == 100
#   * criterion 매개변수 : 불순도 지정. 기본값 == 'gini', entroy 불순도 사용 가능
#   * max_depth : 트리가 성장할 최대 깊이. 기본값 == None
#   * min_samples_split : 노드를 나누기 위한 최소 샘플 개수, 기본값 == 2
#   * max_features 매개변수 : 최적의 분할을 위해 탐색할 특성의 개수 , 기본값 auto , 특성 개수의 제곱근
#   * bootstrap 매개변수 : 부트스트랩 샘플을 사용할지 지정 기본값 == True
#   * oob_score : OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정. 기본값 ==False
#   * n_jobs= 매개변수 : 병렬 실행에 사용할 cpu 코어 수 지정

# ** ExtraTreesClassifier 엑스트라 트리 분류 클래스
#   * RandomForestClassifier와 매개변수 동일

# ** GradientBoostingClassifier 그레이디언트 부스팅 분류 클래스
#   * loss 매개변수 : 손실함수 지정. / 기본값 == 'deviance' == 로지스틱손실함수의미
#   * learning_rate : 트리가 앙상블에 기여하는 정도 조절 기본값 ==0.1
#   * n_estimators 매개변수 : 사용할 훈련 세트의 샘플 비율 지정. 기본값 1.0
#   * subsample 매개변수 : 개별 회귀 트리의 최대 깊이. 기본값 3

# ** HistGradientBoostingClassifier 히스토그램 기반 그레이디언트 부스팅 분류 클래스
#   * learning_rate 매개변수 학습률 또는 감쇠율
#   * 기본값 0.1, 1.0.이면 감쇠가 전혀 없음.
#   * max_iter 부스팅 단계를 수행하는 트리의 개수. 기본값은 100
#   * max_bins 입력데이터를 나눌 구간의 개수. 기본값 ==255,  이보다 크게 지정 불가능
#     1개의 구간이 누락된 값을 위해 추가됨.

확인 문제

In [ ]:
1. 여러 개의 모델을 훈련시키고 각 모델의 예측을 취합하여 최종 결과를 만드느 학습 방식
 앙상블
 

In [ ]:
2. 다음 중 비정형 데이터에 속하는 것
4. 이미지 데이터

In [ ]:
3. 다음 알고리즘 중 기본적으로 부트스트램 샘플을 사용하는 알고리즘?
1.랜덤 포레스트
2.엑스트라 트리
3.그레이디언트 부스팅 >>>>
4. 히스토그램 기반 그레이디언트 부스팅 >>>
 3,4 둘 다 아닌